In [ ]:
# Open the file
from collections import defaultdict, deque, Counter, namedtuple
from functools import reduce, cache, cmp_to_key
from heapq import heappush, heappop, heapify
from itertools import permutations, combinations, product
from math import gcd, sqrt, factorial, ceil, floor
from sortedcontainers import SortedDict, SortedList, SortedSet
from pyrival import * # python competitive programming library
from typing import TypeAlias
from math import prod
import re
import numpy as np
import sys
import pyomo.environ as pymo
import pyomo.opt as pyopt
sys.setrecursionlimit(10000)
def scan_nums(line):
    # Use a regular expression to find numbers
    numbers = re.findall(r'-?\d+\.?\d*', line)
    # Convert to appropriate numerical types
    numbers = [float(num) if '.' in num else int(num) for num in numbers]
    return numbers

def get_blocks(input):
    blocks = []
    cur_block = []
    for line in input:
        if line == '\n':
            blocks.append(cur_block)
            cur_block = []
        else:
            cur_block.append(parse_button(line.strip()))    
    blocks.append(cur_block)
    return blocks
        
input_file = 'input.txt'
test_file = 'test.txt'

with open(input_file, 'r') as file:
    lines  = file.readlines()
    robots_locs = []
    robot_velocity = []
    for i, line in enumerate(lines):
        data = scan_nums(line)
        robots_locs.append((data[1], data[0]))
        robot_velocity.append((data[2], data[3]))
        


def get_final_loc(robots_locs, robot_velocity, R, C, time):
    r, c  = robots_locs
    dc, dr = robot_velocity
    
    
    r = (r + dr * time) % R
    c = (c + dc * time) % C
    return r, c

def get_quadrant_sets(R, C):
    # we will return sets that define the cells of each quadrant
    quad_1 = set()
    quad_2 = set()
    quad_3 = set()
    quad_4 = set()
    mid_r = R // 2
    mid_c = C // 2
    for r in range(R):
        for c in range(C):
            if r < mid_r and c < mid_c:
                quad_1.add((r, c))
            elif r < mid_r and c > mid_c:
                quad_2.add((r, c))
            elif r > mid_r and c < mid_c:
                quad_3.add((r, c))
            elif r > mid_r and c > mid_c:
                quad_4.add((r, c))
    return quad_1, quad_2, quad_3, quad_4

def display_grid(R, C, final_positions):
    grid = [['.' for _ in range(C)] for _ in range(R)]
    for r, c in final_positions:
        grid[r][c] = str(final_positions[(r, c)])
    
    for c in range(C):
        grid[R // 2][c] = ' '
    for r in range(R):
        grid[r][C // 2] = ' '
    for row in grid:
        print(''.join(row))

def get_safety_score(robots_locs, robot_velocity, R, C, t):
    quad_1, quad_2, quad_3, quad_4 = get_quadrant_sets(R, C)
    quad_counts = [0, 0, 0, 0]
    final_positions = defaultdict(int)
    for loc, velocity in zip(robots_locs, robot_velocity):
        # get the final location
        r_f, c_f = get_final_loc(loc, velocity, R, C,  t)
        final_positions[(r_f, c_f)] += 1
        for i, quad in enumerate([quad_1, quad_2, quad_3, quad_4]):
            if (r_f, c_f) in quad:
                quad_counts[i] += 1
    display_grid(R, C, final_positions)
    return prod(quad_counts)
directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]

def connected_components(robots_locs, R, C):
    cluster_counts = 0
    for r in range(R):
        for c in range(C):
            visited = set()
            if (r, c) in robots_locs:
                q = deque([(r, c)])
                while len(q) > 0:
                    r, c = q.popleft()
                    if (r, c) in visited:
                        continue
                    visited.add((r, c))
                    for dr, dc in directions:
                        nr, nc = r + dr, c + dc
                        if 0 <= nr < R and 0 <= nc < C and (nr, nc) not in visited and (nr, nc) in robots_locs:
                            q.append((nr, nc))
                cluster_counts += 1
    return cluster_counts
                
        
R_input, C_input = 103, 101
R_test, C_test = 7, 11

# note since bounds are prime p, q. need p * q time units until board returns to original state.
#print(get_safety_score(robots_locs, robot_velocity, R_test, C_test, 100))
print(get_safety_score(robots_locs, robot_velocity, R_input, C_input, R_input * C_input)) 
        



In [ ]:
def update_robots(robots_locs, robot_velocity, R, C):
    new_robots_locs = []
    for loc, velocity in zip(robots_locs, robot_velocity):
        # get the final location
        r_f, c_f = get_final_loc(loc, velocity, R, C,  1)
        new_robots_locs.append((r_f, c_f))
    return new_robots_locs
def display_robots(robots_locs, R, C):
    grid = [['.' for _ in range(C)] for _ in range(R)]
    print(len(grid), len(grid[0]))
    for r, c in robots_locs:
        grid[r][c] = 'X'
    for row in grid:
        print(''.join(row))
print(robots_locs)
def display_robots_evolution(robots_locs, robot_velocity, R, C, max_t):
    print('initial state')
    display_robots(robots_locs, R, C)
    print('----------------------------------------')
    print(f'----------------- start---------------')
    print('----------------------------------------')
    for t in range(max_t):
        robots_locs = update_robots(robots_locs, robot_velocity, R, C)
        display_robots(robots_locs, R, C)
        print('----------------------------------------')
        print(f'-----------------{ t = }---------------')
        print('----------------------------------------')
        
def display_final_state(robots_locs, robot_velocity, R, C, t):
    final_positions = set()
    for loc, vel in zip(robots_locs, robot_velocity):
        r_f, c_f = get_final_loc(loc, vel, R, C, t)
        final_positions.add((r_f, c_f))
    display_robots(final_positions, R, C)
    
def get_smallest_cluster(robots_locs, R, C):
    max_t = R * C
    heap = []
    max_heap_size = 20
    heapify(heap)
    for t in range(1, max_t + 1):
        print(t)
        cur_clusters = connected_components(robots_locs, R, C)
        if len(heap) < max_heap_size:
            heappush(heap, (-cur_clusters, t))
            
        else:
            max_clusters = -heap[0][0]
            if cur_clusters < max_clusters:
                heappop(heap)
                heappush(heap, (-cur_clusters, t))
    ans = [(- v, t) for v, t in heap]
    for v, t in ans:
        print(f'with {v} connect components we see the final state at time {t = }', t)
        display_final_state(robots_locs, robot_velocity, R, C, t)

    return

get_smallest_cluster(robots_locs, R_input, C_input)

    
        

In [69]:
with open(test_file, 'r') as file:
    lines  = file.readlines()
    robots_locs = []
    robot_velocity = []
    for i, line in enumerate(lines):
        data = scan_nums(line)
        robots_locs.append((data[0], data[1]))
        robot_velocity.append((data[2], data[3]))


In [ ]:
101 * 103

In [ ]:
display_robots_evolution(robots_locs, robot_velocity, R_input, C_input, R_input * C_input)